In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = (high/low)**2
    
    
    
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, pbias3, pred_day, labels, mae, rmse, R, (high/low)

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'yeong'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:

run_num = range(len(folder))

#run_num = [0, 1]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2013.xlsx', '나주_2014.xlsx', '나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['용봉_2013.xlsx', '용봉_2014.xlsx', '용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2013.xlsx', '우치_2014.xlsx', '우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2013.xlsx
data/yeong/자동/나주_2014.xlsx
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-6-4 23:00
data/yeong/자동/용봉_2013.xlsx
data/yeong/자동/용봉_2014.xlsx
data/yeong/자동/용봉_2015.xlsx
data/yeong/자동/용봉_2016.xlsx
data/yeong/자동/용봉_2017.xlsx
data/yeong/자동/용봉_2018.xlsx
data/yeong/자동/용봉_2019.xlsx
data/yeong/자동/용봉_2020.xlsx
t

1/1 [==============================] - 0s 20ms/step - gen_loss: 11.3900 - disc_loss: 0.3665 - rmse: 0.4120 - val_loss: 0.4282
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.4164 - disc_loss: 0.3675 - rmse: 0.4144 - val_loss: 0.4577
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.8223 - disc_loss: 0.3720 - rmse: 0.4053 - val_loss: 0.4009
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.0637 - disc_loss: 0.3593 - rmse: 0.4419 - val_loss: 0.4293
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.3012 - disc_loss: 0.3603 - rmse: 0.4336 - val_loss: 0.4513
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.1420 - disc_loss: 0.3433 - rmse: 0.4692 - val_loss: 0.4325
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.0669 - disc_loss: 0.3677 - rmse: 0.4428 - val_loss: 0.4367
Epoch 56/2000
1/1 [================

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9556 - disc_loss: 0.2961 - rmse: 0.3385 - val_loss: 0.3523
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4080 - disc_loss: 0.3029 - rmse: 0.3516 - val_loss: 0.3629
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5519 - disc_loss: 0.2655 - rmse: 0.3570 - val_loss: 0.3593
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5736 - disc_loss: 0.2676 - rmse: 0.3084 - val_loss: 0.3323
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3896 - disc_loss: 0.2720 - rmse: 0.3628 - val_loss: 0.3478
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1745 - disc_loss: 0.2724 - rmse: 0.3722 - val_loss: 0.3511
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3349 - disc_loss: 0.2731 - rmse: 0.3562 - val_loss: 0.3745
Epoch 114/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0274 - disc_loss: 0.2466 - rmse: 0.2755 - val_loss: 0.3320
Epoch 166/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.0436 - disc_loss: 0.2444 - rmse: 0.3425 - val_loss: 0.3263
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5691 - disc_loss: 0.2717 - rmse: 0.3380 - val_loss: 0.3012
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8610 - disc_loss: 0.2500 - rmse: 0.2959 - val_loss: 0.3309
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6796 - disc_loss: 0.2346 - rmse: 0.3106 - val_loss: 0.3288
Epoch 170/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.6523 - disc_loss: 0.2471 - rmse: 0.3193 - val_loss: 0.3178
Epoch 171/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.8128 - disc_loss: 0.2533 - rmse: 0.3900 - val_loss: 0.3573
Epoch 172/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9822 - disc_loss: 0.2452 - rmse: 0.2791 - val_loss: 0.2984
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3336 - disc_loss: 0.2206 - rmse: 0.3032 - val_loss: 0.3477
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8610 - disc_loss: 0.2212 - rmse: 0.3241 - val_loss: 0.3908
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1378 - disc_loss: 0.2406 - rmse: 0.3301 - val_loss: 0.2922
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8021 - disc_loss: 0.2420 - rmse: 0.3340 - val_loss: 0.3261
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1280 - disc_loss: 0.2335 - rmse: 0.3002 - val_loss: 0.3110
Epoch 229/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.0237 - disc_loss: 0.2266 - rmse: 0.3365 - val_loss: 0.4227
Epoch 230/2000
1/1 [===============

...

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.2803 - disc_loss: 0.1748 - rmse: 0.2535 - val_loss: 0.2837
Epoch 1094/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6276 - disc_loss: 0.1682 - rmse: 0.2522 - val_loss: 0.2718
Epoch 1095/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5567 - disc_loss: 0.1637 - rmse: 0.2453 - val_loss: 0.2621
Epoch 1096/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3882 - disc_loss: 0.1654 - rmse: 0.2624 - val_loss: 0.2734
Epoch 1097/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4070 - disc_loss: 0.1810 - rmse: 0.2995 - val_loss: 0.2524
Epoch 1098/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5218 - disc_loss: 0.1705 - rmse: 0.2591 - val_loss: 0.2673
Epoch 1099/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6642 - disc_loss: 0.1768 - rmse: 0.2765 - val_loss: 0.2846
Epoch 1100/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4945 - disc_loss: 0.1861 - rmse: 0.2702 - val_loss: 0.2865
Epoch 1152/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5123 - disc_loss: 0.1663 - rmse: 0.2691 - val_loss: 0.2789
Epoch 1153/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0895 - disc_loss: 0.1680 - rmse: 0.3138 - val_loss: 0.2965
Epoch 1154/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.7671 - disc_loss: 0.1722 - rmse: 0.2799 - val_loss: 0.2754
Epoch 1155/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6948 - disc_loss: 0.1668 - rmse: 0.2865 - val_loss: 0.2504
Epoch 1156/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3852 - disc_loss: 0.1712 - rmse: 0.2936 - val_loss: 0.3195
Epoch 1157/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6187 - disc_loss: 0.1723 - rmse: 0.2521 - val_loss: 0.2591
Epoch 1158/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5685 - disc_loss: 0.1681 - rmse: 0.2388 - val_loss: 0.2947
Epoch 1210/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2831 - disc_loss: 0.1592 - rmse: 0.3400 - val_loss: 0.2580
Epoch 1211/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.6023 - disc_loss: 0.1670 - rmse: 0.2585 - val_loss: 0.2713
Epoch 1212/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4821 - disc_loss: 0.1739 - rmse: 0.2753 - val_loss: 0.2782
Epoch 1213/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8346 - disc_loss: 0.1712 - rmse: 0.2836 - val_loss: 0.3117
Epoch 1214/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1660 - disc_loss: 0.1676 - rmse: 0.3210 - val_loss: 0.2891
Epoch 1215/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2869 - disc_loss: 0.1540 - rmse: 0.3034 - val_loss: 0.2991
Epoch 1216/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1239 - disc_loss: 0.1600 - rmse: 0.2913 - val_loss: 0.3040
Epoch 1268/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7440 - disc_loss: 0.1657 - rmse: 0.2542 - val_loss: 0.3346
Epoch 1269/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8670 - disc_loss: 0.1724 - rmse: 0.2641 - val_loss: 0.2388
Epoch 1270/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4288 - disc_loss: 0.1757 - rmse: 0.2468 - val_loss: 0.3180
Epoch 1271/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8129 - disc_loss: 0.1653 - rmse: 0.2790 - val_loss: 0.2550
Epoch 1272/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5798 - disc_loss: 0.1561 - rmse: 0.2653 - val_loss: 0.3234
Epoch 1273/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6740 - disc_loss: 0.1633 - rmse: 0.2544 - val_loss: 0.2857
Epoch 1274/2000
1/1 [========

1/1 [==============================] - 0s 4ms/step - loss: 0.2839
MissData :  save/  miss :  (13696, 12)
interpol flag :  [True, False]
folder :  data/yeong/총량/
colum_idx :  :,[28,30,31,36,40,43,44]
file_names[idx] :  [['문평천_2013.xlsx', '문평천_2014.xlsx', '문평천_2015.xlsx', '문평천_2016.xlsx', '문평천_2017.xlsx', '문평천_2018.xlsx', '문평천_2019.xlsx', '문평천_2020.xlsx'], ['만봉천_2013.xlsx', '만봉천_2014.xlsx', '만봉천_2015.xlsx', '만봉천_2016.xlsx', '만봉천_2017.xlsx', '만봉천_2018.xlsx', '만봉천_2019.xlsx', '만봉천_2020.xlsx'], ['장성천2_2013.xlsx', '장성천2_2014.xlsx', '장성천2_2015.xlsx', '장성천2_2016.xlsx', '장성천2_2017.xlsx', '장성천2_2018.xlsx', '장성천2_2019.xlsx', '장성천2_2020.xlsx'], ['광산_2013.xlsx', '광산_2014.xlsx', '광산_2015.xlsx', '광산_2016.xlsx', '광산_2017.xlsx', '광산_2018.xlsx', '광산_2019.xlsx', '광산_2020.xlsx'], ['평동천_2013.xlsx', '평동천_2014.xlsx', '평동천_2015.xlsx', '평동천_2016.xlsx', '평동천_2017.xlsx', '평동천_2018.xlsx', '평동천_2019.xlsx', '평동천_2020.xlsx'], ['우치_2013.xlsx', '우치_2014.xlsx', '우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.x

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [64]:
print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(65088, 186)
tn


In [65]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (45561, 186) val_df.shape :  (13018, 186) test_df.shape: (6509, 186)


In [66]:
#fake_df.shape[0]/8760

In [67]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [68]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  5
out_num_features :  1


In [69]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [70]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

/home/jaewon/work/venv/water-quality-venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/yeong/nitrogen/


In [73]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [74]:
rnn_epochs, rnn_in_setps

(20, 240)

In [75]:
'''
sub_flag = True
history_nse = []

bob = 0
b = 3

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    nse, _, _, _ = compa(
        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -7):
        b = 3
    if(bob > 7):
        b = -3
'''
    


'\nsub_flag = True\nhistory_nse = []\n\nbob = 0\nb = 3\n\nwhile sub_flag:\n    \n\n    \n    gru_model = model_gru(\n        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,\n        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,\n        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)\n    nse, _, _, _ = compa(\n        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,\n        #predict_day = rnn_predict_day)\n        predict_day = 4)\n    \n    if(nse > 0.5):\n        sub_flag = False\n   \n    \n    history_nse.append(nse)\n    \n    print(\'--------------------------\')\n    print(history_nse)\n    print(nse)\n    print(\'--------------------------\')\n    \n        \n    bob += b\n    if(bob < -7):\n        b = 3\n    if(bob > 7):\n        b = -3\n'

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 10,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
10/10 [==============================] - 44s 4s/step - loss: 0.4863 - mean_absolute_error: 0.5283 - nse: 0.0408 - val_loss: 0.3338 - val_mean_absolute_error: 0.4694 - val_nse: 0.6060
Epoch 2/15
10/10 [==============================] - 40s 4s/step - loss: 0.2837 - mean_absolute_error: 0.4031 - nse: 0.4768 - val_loss: 0.2611 - val_mean_absolute_error: 0.4055 - val_nse: 0.6912
Epoch 3/15
10/10 [==============================] - ETA: 0s - loss: 0.1817 - mean_absolute_error: 0.3286 - nse: 0.6241

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)

print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)



In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

In [41]:
#train_df.shape, val_df.shape, test_df.shape

In [42]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
